In [1]:
# make the current working directory the project root directory
import os
PROJECT_ROOT_DIRECTORY = 'senior_project'
while os.path.basename(os.getcwd()) != PROJECT_ROOT_DIRECTORY:
    os.chdir(os.pardir)
print('Current working directory:', os.getcwd())

Current working directory: c:\Users\pavan\Desktop\senior_project


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import yfinance as yf
from environments.discrete_env_v1 import PortfolioManagementEnv

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import risk_matrix
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import plotting

In [19]:
returns_matrix = pd.read_csv('data/dow_10_returns.csv', index_col=0, parse_dates=True)[['AXP_RETURN', 'AAPL_RETURN', 'GS_RETURN']]
returns_matrix.head()

,AXP_RETURN,AAPL_RETURN,GS_RETURN
2010-01-04,0.000000,0.000000,0.000000
2010-01-05,-0.002199,0.001729,0.017680
2010-01-06,0.016165,-0.015906,-0.010674
2010-01-07,0.016219,-0.001849,0.019568
2010-01-08,-0.000715,0.006649,-0.018911


In [20]:
(mean_historical_return(returns_matrix, returns_data=True) == (1+returns_matrix).prod()**(252/len(returns_matrix))-1).all()

True

In [21]:
(risk_matrix(returns_matrix, returns_data=True) == returns_matrix.cov() * 252).all().all()

True

In [13]:
weights = np.array([0.5, 0.2, 0.3])

## Return

### Geometric Means

In [75]:
(1 + returns_matrix @ weights).prod() ** (252 / len(returns_matrix)) - 1

0.14497278693214555

In [150]:
((1 + returns_matrix).prod() ** (252 / len(returns_matrix)) - 1) @ weights

0.13332070045131778

In [162]:
# does this even make sense?
((1 + returns_matrix).prod() @ weights) ** (252 / len(returns_matrix)) - 1

0.16047154087244575

In [185]:
np.exp((weights * np.log((1 + returns_matrix).prod() ** (252 / len(returns_matrix)) - 1)).sum())

0.11977395967852718

### Arithmatic Means

In [161]:
mean_historical_return(returns_matrix, returns_data=True, compounding=False) @ weights

0.16614255293509264

In [154]:
(returns_matrix @ weights).mean() * 252

0.16614255293509267

In [157]:
returns_matrix.mean() @ weights * 252

0.16614255293509264

## Risk

In [83]:
((returns_matrix @ weights).var() * 252) ** 0.5

0.24818389130405208

In [115]:
sigma = 0
for i in range(3):
    for j in range(3):
        sigma += weights[i] * weights[j] * returns_matrix.cov().iloc[i,j]
sigma = (sigma*252)**0.5
sigma

0.24818389130405205